# Incremental Import

So far we’ve covered use cases where you had to transfer an entire table’s contents from the database into Hadoop as a one-time operation. What if you need to keep the imported data on Hadoop in sync with the source table on the relational database side? While you could obtain a fresh copy every day by reimporting all data, that would not be optimal. The amount of time needed to import the data would increase in proportion to the amount of additional data appended to the table daily. This would put an unnecessary performance burden on your database. Why reimport data that has already been imported? For transferring deltas of data, Sqoop offers the ability to do incremental imports.

## Importing Only New Data
### Problem

You have a database table with an INTEGER primary key. You are only appending new rows, and you need to periodically sync the table’s state to Hadoop for further processing.
### Solution

Activate Sqoop’s incremental feature by specifying the --incremental parameter. The parameter’s value will be the type of incremental import. When your table is only getting new rows and the existing ones are not changed, use the append mode.

Incremental import also requires two additional parameters: --check-column indicates a column name that should be checked for newly appended data, and --last-value contains the last value that successfully imported into Hadoop.

The following example will transfer only those rows whose value in column id is greater than 1:

### Discussion

Incremental import in append mode will allow you to transfer only the newly created rows. This saves a considerable amount of resources compared with doing a full import every time you need the data to be in sync. One downside is the need to know the value of the last imported row so that next time Sqoop can start off where it ended. Sqoop, when running in incremental mode, always prints out the value of the last imported row. This allows you to easily pick up where you left off. The following is sample output printed out when doing incremental import in append mode:

### Caution

Any changed rows that were already imported from previous runs won’t be transmitted again. This method is meant for tables that are not updating rows.
Incrementally Importing Mutable Data
### Problem

While you would like to use the incremental import feature, the data in your table is also being updated, ruling out use of the append mode.
### Solution

Use the lastmodified mode instead of the append mode. For example, use the following command to transfer rows whose value in column last_update_date is greater than 2013-05-22 01:01:01:

### Discussion

The incremental mode lastmodified requires a column holding a date value (suitable types are date, time, datetime, and timestamp) containing information as to when each row was last updated. Sqoop will import only those rows that were updated after the last import. This column should be populated to the current time on every new row insertion or on a change to an existing row. This ensures that Sqoop can pick up changed rows accurately. Sqoop knows only what you tell it. The onus is on your application to reliably update this column on every row change. Any row that does not have a modified column, as specified in the --check-column parameter, won’t be imported.

Internally, the lastmodified incremental import consists of two standalone MapReduce jobs. The first job will import the delta of changed data similarly to normal import. This import job will save data in a temporary directory on HDFS. The second job will take both the old and new data and will merge them together into the final output, preserving only the last updated value for each row.

As in the case of the append type, all you need to do for subsequent incremental imports is update the value of the --last-value parameter. For convenience, it is printed out by Sqoop on every incremental import execution.

### Preserving the Last Imported Value
### Problem

Incremental import is a great feature that you’re using a lot. Shouldering the responsibility for remembering the last imported value is getting to be a hassle.
### Solution

You can take advantage of the built-in Sqoop metastore that allows you to save all parameters for later reuse. You can create a simple incremental import job with the following command:



And start it with the --exec parameter:

### Discussion

The Sqoop metastore is a powerful part of Sqoop that allows you to retain your job definitions and to easily run them anytime. Each saved job has a logical name that is used for referencing. You can list all retained jobs using the --list parameter:

You can remove the old job definitions that are no longer needed with the --delete parameter, for example:

And finally, you can also view content of the saved job definitions using the --show parameter, for example:

### Storing Passwords in the Metastore
### Problem

You like the built-in Sqoop metastore for its ability to store jobs and the option to elegantly run them at your convenience. As a next step, you would like to automate the process and start the jobs automatically. Unfortunately, each execution requires you to enter a password, which is not easily automated by a script.
### Solution

Sqoop offers two ways to run jobs from within the metastore without requiring any user input. The first and more secure method is by using the parameter --password-file to pass in the file containing the password. The second, less secure method is to set the property sqoop.metastore.client.record.password in the sqoop-site.xml to true:

### Discussion

Both available methods have their advantages and disadvantages. While using the password file is considered safer, in order to secure it, you need to restrict access to the password file. The Sqoop job will be executed with the permissions of the user running the --exec operation rather than the user who created the saved job. You might need to share the file between the two users.

The second method of storing the password inside the metastore is less secure. The metastore is unencrypted, and thus anyone can easily retrieve your saved password. This method might be feasible if you have a dedicated machine with very restricted user access.
Overriding the Arguments to a Saved Job
### Problem

You have a saved job that has been running fine. Recently it has become slower than usual. You would like to get more details about the execution by adding the --verbose parameter.
### Solution

You can add or override any parameters of the saved job when executing it. All you need to do is add an extra -- after the --exec command, followed by any additional parameters you would like to add. For example, use the following command to add the --verbose parameter to the saved job visits:

### Discussion

Saved jobs can be customized at execution time. This functionality is not limited to adding new parameters like --verbose (used to get more insight into what the job is doing). You can override any arbitrary parameter to check how the job with the new settings will behave without modifying the saved job itself.

Another handy use case is to temporarily change the destination in HDFS or in the Hive table if you need an extra import of data to do some unscheduled investigation or analysis.
Caution

You need to be careful about changing the parameters of saved incremental jobs. Sqoop will always retain the value of the last imported row into the metastore regardless of whether you are customizing the execution or not. Using the saved job to just temporarily dump the data somewhere else might lead to data loss in the main destination.

### Free-Form Query Import

The previous chapters covered the use cases where you had an input table on the source database system and you needed to transfer the table as a whole or one part at a time into the Hadoop ecosystem. This chapter, on the other hand, will focus on more advanced use cases where you need to import data from more than one table or where you need to customize the transferred data by calling various database functions.

### Importing Data from Two Tables
### Problem

You need to import one main table; however, this table is normalized. The important values are stored in the referenced dictionary tables, and the main table contains only numeric foreign keys pointing to the values in the dictionaries rather than to natural keys as in the original cities table. You would prefer to resolve the values prior to running Sqoop and import the real values rather than the numerical keys for the countries.
### Solution

Instead of using table import, use free-form query import. In this mode, Sqoop will allow you to specify any query for importing data. Instead of the parameter --table, use the parameter --query with the entire query for obtaining the data you would like to transfer.

Let’s look at an example with the normalized table normcities and its dictionary countries. In order to achieve the same output as with importing the denormalized table cities, you could use the following Sqoop command:

### Discussion

The free-form query import is one of the advanced features of Sqoop. As with all advanced software features, it gives you great power. With great power comes significant responsibility.

There is a lot to be aware of when using free-form query imports. By using query imports, Sqoop can’t use the database catalog to fetch the metadata. This is one of the reasons why using table import might be faster than the equivalent free-form query import. Also, you have to manually specify some additional parameters that would otherwise be populated automatically. In addition to the --query parameter, you need to specify the --split-by parameter with the column that should be used for slicing your data into multiple parallel tasks. This parameter usually automatically defaults to the primary key of the main table. The third required parameter is --target-dir, which specifies the directory on HDFS where your data should be stored.

#### Caution
The free-form query import can’t be used in conjunction with the \--warehouse-dir parameter.

Sqoop performs highly efficient data transfers by inheriting Hadoop’s parallelism. To help Sqoop split your query into multiple chunks that can be transferred in parallel, you need to include the 
$CONDITIONS
placeholder in the where clause of your query. Sqoop will automatically substitute this placeholder with the generated conditions specifying which slice of data should be transferred by each individual task. While you could skip 

$CONDITIONS 
by forcing Sqoop to run only one job using the --num-mappers 1 parameter, such a limitation would have a severe performance impact.

Sqoop will concurrently run several instances of your query at the same time for different slices of data. With one straightforward join, this won’t be an issue, but it can be an issue for more complex queries.
#### Note

If your query needs more than a few seconds in order to start sending data, it might not be suitable for the free-form query import. If this is the case, you can always run the expensive query once prior to Sqoop import and save its output in a temporary table. Then you can use table import to transfer the data into Hadoop.
### Using Custom Boundary Queries
### Problem

You found free-form query import to be very useful for your use case. Unfortunately, prior to starting any data transfer in MapReduce, Sqoop takes a long time to retrieve the minimum and maximum values of the column specified in the --split-by parameter that are needed for breaking the data into multiple independent tasks.
### Solution

You can specify any valid query to fetch minimum and maximum values of the --split-by column using the --boundary-query parameter:

### Discussion

In order to partition data into multiple independent slices that will be transferred in a parallel manner, Sqoop needs to find the minimum and maximum value of the column specified in the --split-by parameter. In a table-based import, Sqoop uses the table’s primary key by default and generates the query select min(col), max(col) from tbl (for table tbl and split column col). In the case of the free-form query import, there is no table that Sqoop can use for fetching those values; instead, it will use the entire query specified on the command line as a subquery in place of the table name, resulting in a query select min(col), max(col) from ($YOUR_QUERY). Such a query is highly inefficient, as it requires materialization of the output result set prior to moving any data just for the purpose of getting the import boundaries.

Without understanding your query and the underlying data, there aren’t many optimizations that Sqoop can automatically apply. Sqoop does offer the parameter --boundary-query, with which a custom query can override the generated query. The only requirement for this query is to return exactly one row with exactly two columns. The first column will be considered the lower bound, while the second column will be the upper bound. Both values are inclusive and will be imported. The type of both columns must be the same as the type of the column used in the --split-by parameter. Knowing your data and the purpose of your query allows you to easily identify the main table, if there is one, and select the boundaries from this table without any additional join or data transformations.

The query used for fetching boundaries can indeed be arbitrary. Let’s walk through a few examples. If you happen to know the boundaries prior to running Sqoop, you can select them directly without opening a single table using a constant boundary query like SELECT 1, 500. If you’re storing the minimum and maximum values in different tables for accounting purposes, you can fetch the data from there as well. There is no requirement to reference any table used in the --query parameter inside the --boundary-query parameter. As the output of the boundary query serves as the basis for importing data, it is imperative that the return value not skew the import process.

### Renaming Sqoop Job Instances
### Problem

You run several concurrent free-form query imports from various databases at the same time on your Hadoop cluster. All MapReduce jobs are named QueryResult.jar, so it’s very hard to see which MapReduce job belongs to which imported query.
### Solution

You can use the command-line parameter --mapreduce-job-name to specify the name of the generated MapReduce job. This name will then show up in the JobTracker web UI. To name your job normcities, you would use the following command:

### Discussion

Sqoop follows the default behavior of Hadoop in using the submitted JAR name for the MapReduce job name. In a table import, the JAR is named after the table name, resulting in unique JAR and therefore also MapReduce job names. In the free-form query import case, with no single table involved, Sqoop will use QueryResult as the base name for the JAR. All query imports will look exactly the same on the JobTracker web UI. You can use the --mapreduce-job-name parameter to choose a name for your job.
### Importing Queries with Duplicated Columns
### Problem

You have more than one table that you’re joining in your free-form query. Your Sqoop import is failing with an error message about duplicate columns, similar to the following one:

Imported Failed: Duplicate Column identifier specified: 'id'

### Solution

You might need to use SQL projection to rename columns in the query so that each column in the output result set has a unique name. You can do that using the AS syntax. For example, to import city names from the tables cities and normcities, you can use the following query:

### Discussion

During initial preparation and before submitting the MapReduce job, Sqoop performs several actions. One such action is to fetch metadata about the transferred columns and their associated types. During this step, Sqoop will generate a Java class that contains one attribute for each column that will be named as the column itself. Java attributes must be unique; therefore, all columns in your query must have unique names.

While databases generally enforce unique column names in tables, it is a likely scenario that during a join operation two columns from different tables will have the same name. The output result set then contains two columns with the same name. This is especially problematic if your query selects all columns from all join tables using fragments like select table1.*, table2.*. In this case, you must break the general statement down, name each column separately, and use the AS clause to rename the duplicate columns so that the query will not have duplicate names.